**Bank Statement Synthetic Data Generation.** All categories are based on the existing categories in the Bank of America App and all ranges are based on American averages 2021. Feel free to make a copy and change categories or range values/percentages. Full Walkthrough of how to generate data below.

Features of Synthetic Data Generation

1.   Spending Categories from Bank of America Mobile Insights
2.   Income Categories from Bank of America Mobile Insights
3.   Data Ranges for categories following American Averages in 2021
4.   Data Generation is tunable by category and by date
5.   Tuning allows data generation to mimic real-life profiles i.e retirement profile, student profile, low-income profile, high-income profile, etc.  





In [17]:
#imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import random
from collections import Counter
from math import floor


In [89]:
# Define parameters
spending_categories = ['Home_&_Utilities','Transportation','Groceries','Personal_&_Family_Care','Health','Insurance','Restaurants_&_Dining','Shopping_&_Entertainment','Travel','Cash_Checks_Misc','Giving','Business Expenses','Education','Finance','Uncategorized']
income_categories = ['Paychecks_Salary','Interest','Consulting','Deposits','Expense_Reimbursement','Investment_Income','Other_Income','Retirement_Income','Sales','Services','Wages_Paid']
transaction_types = ['Spending', 'Income']
transaction_columns = ['name', 'amount','category','transaction_type','date']

# Define specific ranges for each spending category monthly: Based on BofA categories and American Averages
spending_category_ranges_monthly = {
    'Home_&_Utilities': (1500, 3000),
    'Transportation': (500, 700),
    'Groceries': (300, 500),
    'Personal_&_Family_Care': (50, 200),
    'Health': (200, 500),
    'Insurance': (100, 500),
    'Restaurants_&_Dining': (200, 500),
    'Shopping_&_Entertainment': (100, 300),
    'Travel': (100, 300),
    'Cash_Checks_Misc': (50, 200),
    'Giving': (10, 100),
    'Business Expenses': (100, 1000),
    'Education': (100, 500),
    'Finance': (10, 50),
    'Uncategorized': (50, 200)
}


# Define specific ranges for each income category monthly: Based on BofA categories and American Averages
income_category_ranges_monthly = {
    'Paychecks_Salary': (3000, 5000),
    'Interest': (10, 50),
    'Consulting': (100, 1000),
    'Deposits': (50, 200),
    'Expense_Reimbursement': (100, 500),
    'Investment_Income': (100, 500),
    'Other_Income': (200, 500),
    'Retirement_Income': (1000, 5000),
    'Sales': (100, 5000),
    'Services': (1000, 5000),
    'Wages_Paid': (100, 500)
}

#Recurring total spending ranges
recurring_spending_ranges = {
    'Home_&_Utilities': (1,1),
    'Transportation': (.6, .8),
    'Groceries': (.5, .8),
    'Personal_&_Family_Care': (.5, .8),
    'Health': (.5, 1),
    'Insurance': (1, 1),
    'Restaurants_&_Dining': (.2, .4),
    'Shopping_&_Entertainment': (.2, .4),
    'Travel': (.5, 1),
    'Cash_Checks_Misc': (.1, 1),
    'Giving': (.1, 1),
    'Business Expenses': (.5, 1),
    'Education': (.5, 1),
    'Finance': (1, 1),
    'Uncategorized': (.1, .3)
}

# Recurring total income ranges
recurring_income_ranges = {
    'Paychecks_Salary': (1, 1),
    'Interest': (1, 1),
    'Consulting': (0.5, 1),
    'Deposits': (0.1, 1),
    'Expense_Reimbursement': (0.1, 1),
    'Investment_Income': (0.5, 0.8),
    'Other_Income': (0.1, 0.5),
    'Retirement_Income': (1, 1),
    'Sales': (0.5, 1),
    'Services': (0.5, 1),
    'Wages_Paid': (1, 1)
}

#Name Bank of recurring spending items
spending_recurring_item_names ={
    'Home_&_Utilities': ['Rent', 'Mortgage'],
    'Transportation': ['Gasoline', 'Public Transportation','Car Payment'],
    'Groceries': ['Target', 'Walmart', 'Whole Foods', 'Kroger'],
    'Personal_&_Family_Care': ['Toiletries', 'Cosmetics'],
    'Health': ['Medical Supplies', 'Prescription Medication'],
    'Insurance': ['Health Insurance', 'Car Insurance'],
    'Restaurants_&_Dining': ['Fast Food', 'Dining Out'],
    'Shopping_&_Entertainment': ['Clothing', 'Electronics', 'Movies', 'Concerts'],
    'Travel': ['Airfare', 'Hotels', 'Rental Cars'],
    'Cash_Checks_Misc': ['ATM Withdrawals', 'Miscellaneous Expenses'],
    'Giving': ['Charitable Donations'],
    'Business Expenses': ['Office Supplies', 'Travel Expenses'],
    'Education': ['Tuition', 'Books', 'School Supplies'],
    'Uncategorized': ['Miscellaneous']
}

spending_random_item_names={
        'Home_&_Utilities': ['Appliances', 'Maintenance', 'Home Decor', 'Utilities'],
        'Transportation': ['Maintenance', 'Parking'],
        'Groceries': ['Produce', 'Dairy', 'Meat', 'Bakery'],
        'Personal_&_Family_Care': ['Skincare', 'Hygiene', 'Baby Care', 'Grooming'],
        'Health': ['Vitamins', 'Wellness', 'Checkup', 'Therapy'],
        'Insurance': ['Coverage', 'Policy', 'Premium', 'Claim'],
        'Restaurants_&_Dining': ['Cuisine', 'Dining', 'Takeout', 'Beverages'],
        'Shopping_&_Entertainment': ['Fashion', 'Electronics', 'Tickets', 'Games'],
        'Travel': ['Vacation', 'Destination', 'Reservations', 'Adventure'],
        'Cash_Checks_Misc': ['Withdrawal', 'Transactions', 'Miscellaneous', 'Fees'],
        'Giving': ['Donation', 'Support', 'Contribution', 'Philanthropy'],
        'Business Expenses': ['Supplies', 'Fees', 'Equipment', 'Services'],
        'Education': ['Courses', 'Books', 'Materials', 'Learning'],
        'Uncategorized': ['Miscellaneous', 'General', 'Other', 'Unspecified']
}

#Name Bank for recurring income items

income_category_names = {
    'Paychecks_Salary': ['Bi-weekly Paycheck'],
    'Interest': ['Savings Account Interest', 'Investment Dividends'],
    'Consulting': ['Contract Work'],
    'Deposits': ['Bank Deposits'],
    'Expense_Reimbursement': ['Travel Expense Reimbursement', 'Work Expense Reimbursement'],
    'Investment_Income': ['Investment Gains', 'Stock Dividends'],
    'Other_Income': ['Bonus', 'Royalties'],
    'Retirement_Income': ['Pension', 'Retirement Account Withdrawal'],
    'Sales': ['Product Sales', 'Service Sales'],
    'Services': ['Professional Services', 'Freelance Work'],
    'Wages_Paid': ['Employee Wages', 'Contractor Payment']
}

In [83]:
#Generate Monthly Spending and Income for Ranges

#Generate definitive amounts for all categories in input map
def generate_definitive_amounts(category_ranges):
    definitive_amounts = {}
    for category, range_tuple in category_ranges.items():
        min_amount, max_amount = range_tuple
        amount = np.random.uniform(min_amount, max_amount)
        definitive_amounts[category] = round(amount)
    return definitive_amounts

#Generate definitive amounts for input categories in input array and in input map. 0 for all other categories
def generate_definitive_amounts_for_categories(categories, category_ranges):
    definitive_amounts = {}
    # First, set all categories from category_ranges to 0
    for category in category_ranges.keys():
        definitive_amounts[category] = 0
    for category in categories:
        if category in category_ranges:
            min_amount, max_amount = category_ranges[category]
            amount = np.random.uniform(min_amount, max_amount)
            definitive_amounts[category] = round(amount)
    return definitive_amounts

#We check if there are items available in the name bank.
#We  select a total amount within the range of defined in recurring_spending_ranges of the definitive amount for the category.
# We then iteratively select items from the name bank and assign random amounts to them, making sure that the total amount of all items does not exceed the total amount selected.
#If there's still available amount after assigning amounts to all items, we distribute it randomly among the items.
#Finally, we create transactions for each item and append them to the recurring_spending list.
def generate_recurring_items(category_amounts, name_bank, recurring_spending_ranges, transaction_type, date_range):
    recurring_spending = []

    for category, amount in category_amounts.items():
        if category in name_bank:
            bank_items = name_bank[category].copy()  # Work off of a local copy of the name_bank
            range_min, range_max = recurring_spending_ranges[category]  # Use the recurring_spending_ranges for the category
            total_amount_range = (amount * range_min, amount * range_max)  # Define the total amount range
            total_amount = random.uniform(*total_amount_range)  # Randomly select a total amount within the range
            available_amount = total_amount

            item_amounts = []
            while available_amount > 0 and bank_items:
                item = random.choice(bank_items)  # Select a random item from the name bank
                bank_items.remove(item)  # Remove the selected item from the name bank
                if available_amount != amount:
                    item_amount = random.uniform(0.01, min(available_amount, amount * 0.5))  # Generate a random amount for the item
                else:
                    item_amount = available_amount
                item_amounts.append((item, floor(item_amount)))  # Append the item and its amount to the list
                available_amount -= item_amount  # Update the available amount

            # Create transactions for each item
            for item, item_amount in item_amounts:
                # Generate a random date within the specified date range
                transaction_date = random.choice(date_range)

                transaction = {
                    "name": item,
                    "amount": item_amount,
                    "category": category,
                    "transaction_type": transaction_type,
                    "date": transaction_date.strftime('%Y-%m-%d')  # Format the date
                }
                recurring_spending.append(transaction)

    return json.dumps(recurring_spending, indent=4)

#TEST
def_amounts = generate_definitive_amounts(spending_category_ranges_monthly)
print(def_amounts)
recurring_spending = generate_recurring_items(def_amounts, spending_recurring_item_names, recurring_spending_ranges,'Spending', pd.date_range(pd.to_datetime('2024-01-01'),pd.to_datetime('2024-01-28')))
print(recurring_spending)

def_amounts_income = generate_definitive_amounts(income_category_ranges_monthly)
print(def_amounts_income)
recurring_income = generate_recurring_items(def_amounts_income, income_category_names, recurring_income_ranges,'Income', pd.date_range(pd.to_datetime('2024-01-01'),pd.to_datetime('2024-01-28')))
print(recurring_income)

{'Home_&_Utilities': 1923, 'Transportation': 526, 'Groceries': 496, 'Personal_&_Family_Care': 70, 'Health': 433, 'Insurance': 374, 'Restaurants_&_Dining': 229, 'Shopping_&_Entertainment': 276, 'Travel': 130, 'Cash_Checks_Misc': 195, 'Giving': 98, 'Business Expenses': 830, 'Education': 450, 'Finance': 20, 'Uncategorized': 87}
[
    {
        "name": "Electricity",
        "amount": 1923,
        "category": "Home_&_Utilities",
        "transaction_type": "Spending",
        "date": "2024-01-27"
    },
    {
        "name": "Car Payment",
        "amount": 104,
        "category": "Transportation",
        "transaction_type": "Spending",
        "date": "2024-01-03"
    },
    {
        "name": "Public Transportation",
        "amount": 97,
        "category": "Transportation",
        "transaction_type": "Spending",
        "date": "2024-01-09"
    },
    {
        "name": "Gasoline",
        "amount": 141,
        "category": "Transportation",
        "transaction_type": "Spending",
  

In [78]:

pd.set_option('display.max_rows', None) #see full table
pd.set_option('display.max_columns', None) #see full table
#definitive_amounts: A dictionary containing definitive amounts for each spending category.
#recurring_payments: JSON data representing recurring payments.
#category_name_map: A mapping of spending categories to their respective names.
#date_range: A range of dates within which transactions will be generated.
#Here's a summary of what the function does:

#Function Summary
#It creates a DataFrame recurring_df from the provided JSON data (recurring_payments) using pd.DataFrame.from_records.
#It extracts the minimum date from the provided date_range and stores the year and month.
#It modifies the date column of recurring_df to match the extracted year and month while preserving the original day.
#It calculates the running sum of each spending category's amount.
#It generates random transactions for each spending category based on the definitive amounts and the remaining amounts to be spent.
#For each transaction, it randomly selects a date within the provided date_range.
#It constructs a list of dictionaries, where each dictionary represents a transaction, containing details such as name, amount, category, transaction type, and date.
#It creates a DataFrame transactions_df from the list of transaction dictionaries.
#It concatenates recurring_df and transactions_df to create a final DataFrame final_df.
#It sorts final_df by the date column to ensure transactions are in chronological order.
#Finally, it returns the sorted DataFrame final_df.
def generate_transactions_with_recurring_payments(definitive_amounts, recurring_payments, category_name_map, date_range):

    # Create a DataFrame for recurring payments
    recurring_df = pd.DataFrame.from_records(json.loads(recurring_payments), columns=transaction_columns)

    # Extract the minimum date from the date range
    min_date = min(date_range)
    min_year, min_month = min_date.year, min_date.month

    # Modify the date column of recurring_df to match the year and month of min_date but keep the original day
    recurring_df['date'] = pd.to_datetime(recurring_df['date'])
    recurring_df['date'] = recurring_df['date'].apply(lambda x: pd.Timestamp(year=min_year, month=min_month, day=x.day))

    # Calculate the running sum of each category's amount
    running_sum = recurring_df.groupby('category')['amount'].sum().to_dict()

    transactions = []
    for category, amount in definitive_amounts.items():
        # Calculate the amount left to generate random transactions for
        remaining_amount = max(amount - running_sum.get(category, 0), 0)

        # Generate random number of transactions between 5 and 10
        num_transactions_per_category = np.random.randint(5, 11)

        # Determine transaction type based on category
        transaction_type = "Spending" if category in spending_categories else "Income"

        # Generate random transactions for the remaining amount
        if remaining_amount > 0:
            # Calculate the amount for each transaction
            transaction_amounts = np.random.uniform(remaining_amount * 0.1, remaining_amount * 0.2, num_transactions_per_category)

            # Convert numpy datetime objects to Python datetime objects
            date_range_python = [date.to_pydatetime() for date in date_range]

            # Generate random dates within the specified date range
            random_dates = [random.choice(date_range_python) for _ in range(num_transactions_per_category)]

            for transaction_amount, transaction_date in zip(transaction_amounts, random_dates):
                if transaction_amount > 0:
                    transaction = {
                        "name": random.choice(category_name_map.get(category, ["Unknown"])),
                        "amount": floor(transaction_amount),
                        "category": category,
                        "transaction_type": transaction_type,
                        "date":  transaction_date.strftime('%Y-%m-%d')  # Fill in the date value
                    }
                    transactions.append(transaction)


    # Create a DataFrame for transactions
    transactions_df = pd.DataFrame.from_records(transactions)

    # Concatenate recurring_df with transactions_df
    final_df = pd.concat([recurring_df, transactions_df], ignore_index=True)

    # Sort the DataFrame by the date column
    final_df['date'] = pd.to_datetime(final_df['date'])
    final_df.sort_values(by='date', inplace=True)

    return final_df


#TEST
test_df_spending = generate_transactions_with_recurring_payments(def_amounts, recurring_spending,spending_random_item_names, pd.date_range(pd.to_datetime('2024-02-01'),pd.to_datetime('2024-02-28')))
print(test_df_spending)
test_df_income = generate_transactions_with_recurring_payments(def_amounts_income, recurring_income,income_category_names,pd.date_range(pd.to_datetime('2024-02-01'),pd.to_datetime('2024-02-28')))
print(test_df_income)

                        name  amount                  category  \
37                   Produce      17                 Groceries   
2      Public Transportation     194            Transportation   
116                  Unknown       6                   Finance   
126              Unspecified      28             Uncategorized   
69                   Fashion      39  Shopping_&_Entertainment   
31                   Parking      38            Transportation   
113                  Unknown       6                   Finance   
14                 Fast Food       5      Restaurants_&_Dining   
17               Electronics       4  Shopping_&_Entertainment   
11   Prescription Medication       0                    Health   
94              Philanthropy      11                    Giving   
96                  Donation       8                    Giving   
48                   Hygiene      11    Personal_&_Family_Care   
21               Rental Cars      51                    Travel   
111       

In [80]:
#Generate Full Statement
def generate_monthly_transactions(start_date, end_date, definitive_amounts_spending, definitive_amounts_income, recurring_payments_spending, recurring_payments_income, category_name_map_spending, category_name_map_income):
    # Convert start_date and end_date to datetime if they are strings
    if isinstance(start_date, str):
        start_date = pd.to_datetime(start_date)
    if isinstance(end_date, str):
        end_date = pd.to_datetime(end_date)

    # Initialize an empty list to store DataFrames for each month
    monthly_dfs = []

    # Loop through each month in the date range
    current_date = start_date
    while current_date <= end_date:
        # Generate transactions for the current month
        transactions_monthly_spending = generate_transactions_with_recurring_payments(definitive_amounts_spending, recurring_payments_spending, category_name_map_spending, pd.date_range(current_date, periods=30))
        # Append the resulting DataFrame to the list
        monthly_dfs.append(transactions_monthly_spending)
        # Generate income for the current month
        transactions_monthly_income = generate_transactions_with_recurring_payments(definitive_amounts_income, recurring_payments_income, category_name_map_income, pd.date_range(current_date, periods=30))
        # Append the resulting DataFrame to the list
        monthly_dfs.append(transactions_monthly_income)
        # Move to the next month
        current_date = current_date + pd.DateOffset(months=1)

    # Concatenate all DataFrames into a single DataFrame
    monthly_df = pd.concat(monthly_dfs, ignore_index=True)

    # Sort the DataFrame by the date column
    monthly_df['date'] = pd.to_datetime(monthly_df['date'])
    monthly_df.sort_values(by='date', inplace=True)
    return monthly_df



In [92]:
#Generate Full Statement and Export to CSV Workflow

#Note: Full Categories for spending and income below for reference in tuning
#spending_categories = ['Home_&_Utilities','Transportation','Groceries','Personal_&_Family_Care','Health','Insurance','Restaurants_&_Dining','Shopping_&_Entertainment','Travel','Cash_Checks_Misc','Giving','Business Expenses','Education','Finance','Uncategorized']
#income_categories = ['Paychecks_Salary','Interest','Consulting','Deposits','Expense_Reimbursement','Investment_Income','Other_Income','Retirement_Income','Sales','Services','Wages_Paid']

#Step 1: Generate the definitive values for each category range of spending. New values are randomly generated within their specified range each run: Can Tune income_categories for categories you want included from above categories
spending_categories_chosen = ['Home_&_Utilities','Transportation','Groceries','Personal_&_Family_Care','Health','Insurance','Restaurants_&_Dining','Shopping_&_Entertainment','Travel','Finance','Uncategorized']
definitive_spending = generate_definitive_amounts_for_categories(spending_categories_chosen, spending_category_ranges_monthly) #Use function generate_definitive_amounts_for_categories to specify what categories to include (all other categories will be assigned 0)
#OtherWise use generate_definitive_amounts to generate a definitive value for all categories. Not Recommended since using all spending categories can be excessive
#definitive_spending = generate_definitive_amounts(spending_category_ranges_monthly)

#Step 2: Just like step 1 but for income: Can Tune income_categories for categories you want included from above categories
income_categories = ['Paychecks_Salary','Interest','Deposits','Expense_Reimbursement','Investment_Income']
definitive_income = generate_definitive_amounts_for_categories(income_categories, income_category_ranges_monthly)
#definitive_income = generate_definitive_amounts(income_category_ranges_monthly)

#Step 3: Generate Recurring spending: Nothing to Tune here
recurring_spending = generate_recurring_items(definitive_spending, spending_recurring_item_names, recurring_spending_ranges,'Spending', pd.date_range(pd.to_datetime('2024-01-01'),pd.to_datetime('2024-01-28'))) #Date range here is not important since it will be changed later on. Only the generated day will stay the same for recurring spending

#Step 4: Generate Recurring income: Nothing to Tune here
recurring_income = generate_recurring_items(definitive_income, income_category_names, recurring_income_ranges,'Income', pd.date_range(pd.to_datetime('2024-01-01'),pd.to_datetime('2024-01-28'))) #Date range here is not important since it will be changed later on. Only the generated day will stay the same for recurring spending

#Step 5: Generate Full Bank Statement: Can Tune datetime range for less or greater than the current 2024 range. 1 year generates about 2000 rows.
bank_statement = generate_monthly_transactions(pd.to_datetime('2024-01-01'), pd.to_datetime('2024-12-31'), definitive_spending, definitive_income, recurring_spending, recurring_income, spending_random_item_names, income_category_names)
bank_statement.to_csv('bankStatement.csv', index=False)
#Note: Google Colab saves to content file -> must download if you want to upload to folder or save locally